### File for the manipulation check in the online-study

In [ ]:
# library imports
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pickle
# "Extra package"
import pingouin as pg
# Vallat, R. (2018). Pingouin: statistics in Python. Journal of Open Source Software, 3(31), 1026,
# https://doi.org/10.21105/joss.01026

In [ ]:
# import the dataset
with open("OnlineStudy_RawData.json") as jsonData:
    dataset = json.load(jsonData)

In [ ]:
# import the ids after removing bad cases in the typing task
with open("filtered_online_study_ids_taskLvl", "rb") as fp:
    filter_task_lvl = pickle.load(fp)

In [ ]:
# import the sociodemographic dataset
sociodem_df = pd.read_csv("OnlineStudy_sociodemographic_data.csv", index_col="firebase_id")
sociodem_df = sociodem_df.loc[filter_task_lvl, :]

#### --- create a dataframe with the data for the manipulation check ---

In [ ]:
manipulation_check_data = {}

# loop over all datasets
for par in dataset:

    # only include the data of the selected participants
    if par in filter_task_lvl:

        manipulation_check_data[par] = {}

        # MDBF

        # recode some items of the MDBF scale
        items_to_recode = ["MDBF_angespannt", "MDBF_nervoes", "MDBF_schlaefrig", "MDBF_ungluecklich",
                           "MDBF_unzufrieden", "MDBF_ermattet"]

        # get the data for the practice and actual condition

        # Condition
        for item in dataset[par]["Con_Mdbf"]["data"]:
            value = dataset[par]["Con_Mdbf"]["data"][item]
            if item in items_to_recode:
                manipulation_check_data[par]["Con_" + item] = 4 - value
            else:
                manipulation_check_data[par]["Con_" + item] = value

        # Practice
        for item in dataset[par]["Pr_Mdbf"]["data"]:
            value = dataset[par]["Pr_Mdbf"]["data"][item]
            if item in items_to_recode:
                manipulation_check_data[par]["Pr_" + item] = 4 - value
            else:
                manipulation_check_data[par]["Pr_" + item] = value

        # SAM

        for page in dataset[par]:
            if "_Sam_" in page:
                manipulation_check_data[par][page + "_Valenz"] = dataset[par][page]["data"]["samValence"]
                manipulation_check_data[par][page + "_Arousal"] = dataset[par][page]["data"]["samArousal"]

        # Count Task Answers

        count_answer = 0
        count_targets = 0
        for page in dataset[par]:
            if "_Count_" in page:
                targs = dataset[par][page]["data"]["Total_num_targets"]
                manipulation_check_data[par]["count_targets_" + page[9:]] = targs
                count_targets += targs
            if "_CountAns_" in page:
                ans = dataset[par][page]["data"]["Count_Task_Answer"]
                manipulation_check_data[par]["count_answer_" + page[12:]] = ans
                count_answer += ans

        manipulation_check_data[par]["Count_Total_Ans"] = count_answer
        manipulation_check_data[par]["Count_Total_Targets"] = count_targets

        # Add the condition to the Dataset
        manipulation_check_data[par]["condition"] = dataset[par]["ExperimentMetaData"]["condition"]

# create a pandas dataframe with the manipulation check data
man_check_df = pd.DataFrame(manipulation_check_data).T

# create the MDBF scales
man_check_df["Con_MDBF_GS"] = (man_check_df["Con_MDBF_wohl"] + man_check_df["Con_MDBF_gut"] + man_check_df[
    "Con_MDBF_ungluecklich"] + man_check_df["Con_MDBF_unzufrieden"]) / 4
man_check_df["Con_MDBF_RU"] = (man_check_df["Con_MDBF_ausgeglichen"] + man_check_df["Con_MDBF_ruhig"] + man_check_df[
    "Con_MDBF_angespannt"] + man_check_df["Con_MDBF_nervoes"]) / 4
man_check_df["Con_MDBF_WM"] = (man_check_df["Con_MDBF_frisch"] + man_check_df["Con_MDBF_wach"] + man_check_df[
    "Con_MDBF_schlaefrig"] + man_check_df["Con_MDBF_ermattet"]) / 4

man_check_df["Pr_MDBF_GS"] = (man_check_df["Pr_MDBF_wohl"] + man_check_df["Pr_MDBF_gut"] + man_check_df[
    "Pr_MDBF_ungluecklich"] + man_check_df["Pr_MDBF_unzufrieden"]) / 4
man_check_df["Pr_MDBF_RU"] = (man_check_df["Pr_MDBF_ausgeglichen"] + man_check_df["Pr_MDBF_ruhig"] + man_check_df[
    "Pr_MDBF_angespannt"] + man_check_df["Pr_MDBF_nervoes"]) / 4
man_check_df["Pr_MDBF_WM"] = (man_check_df["Pr_MDBF_frisch"] + man_check_df["Pr_MDBF_wach"] + man_check_df[
    "Pr_MDBF_schlaefrig"] + man_check_df["Pr_MDBF_ermattet"]) / 4

# merge all SAM items together to one
man_check_df["Con_valence_merged"] = (man_check_df['Con_Sam_PatternTyping_Valenz'] + man_check_df['Con_Sam_DragDrop_Valenz'] +
                                      man_check_df['Con_Sam_FollowBox_Valenz'] + man_check_df['Con_Sam_PointClick_Valenz'] +
                                      man_check_df['Con_Sam_Slider_Valenz']) / 5

man_check_df["Con_arousal_merged"] = (man_check_df['Con_Sam_PatternTyping_Arousal'] + man_check_df['Con_Sam_DragDrop_Arousal'] +
                                      man_check_df['Con_Sam_FollowBox_Arousal'] + man_check_df['Con_Sam_PointClick_Arousal'] +
                                      man_check_df['Con_Sam_Slider_Arousal']) / 5

man_check_df["Pr_valence_merged"] = (man_check_df['Pr_Sam_PatternTyping_Valenz'] + man_check_df['Pr_Sam_DragDrop_Valenz'] +
                                      man_check_df['Pr_Sam_FollowBox_Valenz'] + man_check_df['Pr_Sam_PointClick_Valenz'] +
                                      man_check_df['Pr_Sam_Slider_Valenz']) / 5

man_check_df["Pr_arousal_merged"] = (man_check_df['Pr_Sam_PatternTyping_Arousal'] + man_check_df['Pr_Sam_DragDrop_Arousal'] +
                                      man_check_df['Pr_Sam_FollowBox_Arousal'] + man_check_df['Pr_Sam_PointClick_Arousal'] +
                                      man_check_df['Pr_Sam_Slider_Arousal']) / 5


# concat the sociodemographic info the the manipultation check dataframe
man_check_df = pd.concat([man_check_df, sociodem_df], axis=1)

# list all manipulation check items
list(man_check_df.columns)

In [ ]:
 count the number of participants per condition
# 0 = high stress, 1 = low stress
print("Number of participants per condition:", "\n", man_check_df["condition"].value_counts())

#### --- Get a list of all variables which are included in the manipulation check ---

In [ ]:
# Variables to test

mdbf = ["MDBF_GS", "MDBF_RU", "MDBF_WM", "stress", "nostalgia"]
sam = ['Sam_PatternTyping_Arousal',
       'Sam_PatternTyping_Valenz',
       'arousal_merged',
       'valence_merged',
       'Sam_DragDrop_Arousal',
       'Sam_DragDrop_Valenz',
       'Sam_FollowBox_Arousal',
       'Sam_FollowBox_Valenz',
       'Sam_PointClick_Arousal',
       'Sam_PointClick_Valenz',
       'Sam_Slider_Arousal',
       'Sam_Slider_Valenz', ]

#### --- Get the descriptive stats of the manipulation check data ---

In [ ]:
# get descriptive statistics of SAM, Count Answers, MDBF answers
descriptive_hs_data = man_check_df.loc[man_check_df["condition"] == 0].drop(["Con_MDBF_wohl", "Con_MDBF_gut",
                                                                             "Con_MDBF_ungluecklich", "Con_MDBF_unzufrieden",
                                                                             "Con_MDBF_ausgeglichen", "Con_MDBF_ruhig", "Con_MDBF_angespannt", "Con_MDBF_nervoes",
                                                                             "Con_MDBF_frisch", "Con_MDBF_wach", "Con_MDBF_schlaefrig", "Con_MDBF_ermattet",
                                                                             "Pr_MDBF_wohl", "Pr_MDBF_gut", "Pr_MDBF_ungluecklich", "Pr_MDBF_unzufrieden",
                                                                             "Pr_MDBF_ausgeglichen", "Pr_MDBF_ruhig", "Pr_MDBF_angespannt", "Pr_MDBF_nervoes",
                                                                             "Pr_MDBF_frisch", "Pr_MDBF_wach", "Pr_MDBF_schlaefrig", "Pr_MDBF_ermattet"
                                                                             ], axis=1).describe().sort_index(axis=1)
descriptive_ls_data = man_check_df.loc[man_check_df["condition"] == 1].drop(["Con_MDBF_wohl", "Con_MDBF_gut", "Con_MDBF_ungluecklich", "Con_MDBF_unzufrieden",
                                                                             "Con_MDBF_ausgeglichen", "Con_MDBF_ruhig", "Con_MDBF_angespannt", "Con_MDBF_nervoes",
                                                                             "Con_MDBF_frisch", "Con_MDBF_wach", "Con_MDBF_schlaefrig", "Con_MDBF_ermattet",
                                                                             "Pr_MDBF_wohl", "Pr_MDBF_gut", "Pr_MDBF_ungluecklich", "Pr_MDBF_unzufrieden",
                                                                             "Pr_MDBF_ausgeglichen", "Pr_MDBF_ruhig", "Pr_MDBF_angespannt", "Pr_MDBF_nervoes",
                                                                             "Pr_MDBF_frisch", "Pr_MDBF_wach", "Pr_MDBF_schlaefrig", "Pr_MDBF_ermattet"
                                                                             ], axis=1).describe().sort_index(axis=1)


#### --- Calculate Cronbachs alpha for the MDBF subscales ---

In [ ]:
cronbachs_alphas = {}

# get the Cronbach's Alpha Values of the MDBF scale
mdbf_scales = {
    "Con_GS": ["Con_MDBF_wohl", "Con_MDBF_gut", "Con_MDBF_ungluecklich", "Con_MDBF_unzufrieden"],
    "Con_RU": ["Con_MDBF_ausgeglichen", "Con_MDBF_ruhig", "Con_MDBF_angespannt", "Con_MDBF_nervoes"],
    "Con_WM": ["Con_MDBF_frisch", "Con_MDBF_wach", "Con_MDBF_schlaefrig", "Con_MDBF_ermattet"],
    "Pr_GS": ["Pr_MDBF_wohl", "Pr_MDBF_gut", "Pr_MDBF_ungluecklich", "Pr_MDBF_unzufrieden"],
    "Pr_RU": ["Pr_MDBF_ausgeglichen", "Pr_MDBF_ruhig", "Pr_MDBF_angespannt", "Pr_MDBF_nervoes"],
    "Pr_WM": ["Pr_MDBF_frisch", "Pr_MDBF_wach", "Pr_MDBF_schlaefrig", "Pr_MDBF_ermattet"]
}

# show and save the cronbachs alpha values per MDBF scale
for i in mdbf_scales:
    cronbachs_alphas[i] = {}
    # get cronbachs alpha for the high-stress and low-stress condition separately
    alpha_hs = pg.cronbach_alpha(data=man_check_df.loc[man_check_df["condition"] == 0].loc[:, mdbf_scales[i]])
    alpha_ls = pg.cronbach_alpha(data=man_check_df.loc[man_check_df["condition"] == 1].loc[:, mdbf_scales[i]])
    print("Cronbachs Alpha HS condition" + str(i), alpha_hs)
    print("Cronbachs Alpha LS condition" + str(i), alpha_ls)
    cronbachs_alphas[i]["HS_alpha"] = alpha_hs[0]
    cronbachs_alphas[i]["HS_alpha_ci"] = alpha_hs[1]
    cronbachs_alphas[i]["LS_alpha"] = alpha_ls[0]
    cronbachs_alphas[i]["LS_alpha_ci"] = alpha_ls[1]

# save the cronbach alphas as a dataframe
cronbachs_alphas_df = pd.DataFrame(cronbachs_alphas).T

### --- Main Manipulation Check analysis using a mixed ANOVA with experimental phase as within-subject factor and condition as between subject factor ---

#### --- helper functions to do the mixed ANOVA ---

In [ ]:
# prepare the dataset for the ANOVA

# helper to split the prepare the dataset for the anova and change the data from a wide to a long format
def create_anova_df(data, variable):
    # assign a subject number to each subject
    data["subject"] = np.arange(len(data))
    # replace the condition number with a string
    data["condition"].replace({0: "HS", 1: "LS"}, inplace=True)

    # change the format from wide to long
    df = pd.melt(data, id_vars=["subject", "condition"], value_vars=["Pr_" + variable, "Con_" + variable],
                 var_name="Pr-Con", value_name=variable)

    # change the name of the column to practice and condition for clearer reading of the results
    df["Pr-Con"].replace({"Pr_" + variable: "Pr", "Con_" + variable: "Con"}, inplace=True)

    return df

In [ ]:
# Create PointPlots to visualize the manipulation check results
# Tutorial here: https://raphaelvallat.com/pingouin.html
def plot_anova(data, variable):

    dataset = create_anova_df(data, variable)
    # data visualization
    sns.set()
    sns.pointplot(data=dataset, x="Pr-Con", y=variable, hue="condition", dodge=True, markers=['o', 's'],
                  capsize=.1, errwidth=1, palette='colorblind')

    plt.title("Pointplot with " + variable)
    # show the plot
    plt.show()
    # save the plot
    # plt.savefig('ANOVA' + variable + '.png')

In [ ]:
# get the anova results

def calc_anova(data, variable):

    # get the dataset
    dataset = create_anova_df(data, variable)
    # data visualization
    aov = pg.mixed_anova(dv=variable, within="Pr-Con", between="condition", subject="subject", data=dataset)
    print("Repeated measures ANOVA with " + variable)
    pg.print_table(aov)
    posthocs = pg.pairwise_ttests(dv=variable, within='Pr-Con', between='condition',
                                  subject='subject', data=dataset, return_desc=True)
    pg.print_table(posthocs)
    print("\n" + "\n")

    # return the anova and post hoc dataframes with an added index layer that is the variable name
    return pd.concat([aov], keys=[variable]), pd.concat([posthocs], keys=[variable])

In [ ]:
# helper function to calc the result dataframes from a variables list
def get_anova_results(data, variable_list):

    # initialize dataframes
    anova_df = pd.DataFrame()
    posthoc_df = pd.DataFrame()

    # loop the variable list and add the result dataframes to the initialized dataframes
    for i in variable_list:

        anova, posthoc = calc_anova(data, i)

        anova_df = pd.concat([anova_df, anova])
        posthoc_df = pd.concat([posthoc_df, posthoc])

    return anova_df, posthoc_df

#### --- Do the ANOVA analysis (speratelty for the MDBF subscales and for the SAM) ---

MDBF

In [ ]:
# 1. plot the anova graphs for the mdbf (use the entire sample)
for i in mdbf:
    plot_anova(man_check_df, i)

In [ ]:
# 2. get the manipulation check anova results for the MDBF
mdbf_anova, mdbf_posthocs = get_anova_results(man_check_df, mdbf)

SAM

In [ ]:
# 1. plot the anova graphs for the sam (use the entire sample)
for i in sam:
    plot_anova(man_check_df, i)

In [ ]:
# 2. get the anova results for the sam
sam_anova, sam_posthocs = get_anova_results(man_check_df, sam)

#### --- Save the results of all calculations in the manipulation check in one excel file ---

In [ ]:
# save the results
# save the anova and post hoc results in an excel file
with pd.ExcelWriter("Online_Study_Manipulation_Check_Results.xlsx") as writer:
    descriptive_hs_data.to_excel(writer, sheet_name="Des_Stats_High_Stress_Condition")
    descriptive_ls_data.to_excel(writer, sheet_name="Des_Stats_Low_Stress_Condition")
    mdbf_anova.to_excel(writer, sheet_name="MDBF_Anova")
    mdbf_posthocs.to_excel(writer, sheet_name="MDBF_Posthocs")
    sam_anova.to_excel(writer, sheet_name="SAM_Anova")
    sam_posthocs.to_excel(writer, sheet_name="SAM_Posthocs")
    cronbachs_alphas_df.to_excel(writer, sheet_name="Cronbachs_Alphas")

print("Successfully saved.")